## Retorno API (NOX Trading)

> "volume": 127.97,
"compra": 20678.21,
"venda": 20903.64,
"ultimo": 20450.0

## Retorno API (TemBTC e NegocieCoins)
> "buy": 21447.14,
"date": 1556399915,
"high": 21732.25,
"last": 21447.15,
"low": 18722.99,
"sell": 21447.15,
"vol": 81045.27180318

### Campos:

> + date: timestamp da última atualização do ticker.  
+ sell: valor atual em Reais ofertado para venda.  
+ buy: valor atual em Reais ofertado para compra.  
+ last: último valor em Reais negociado.  
+ high: maior valor em Reais de negociação das últimas 24 horas.  
+ low: menor valor em Reais de negociação das últimas 24 horas.  
+ vol: volume da moeda negociada nas últimas 24 horas.  

Os valores em high, low e vol são referentes aos acumulado durante um dia de negociação no Brasil (GMT-3).  

In [110]:
import base64, hashlib, hmac, random, time, urllib, json, requests
import pandas as pd

In [111]:
urlApiTemBTC = 'https://broker.tembtc.com.br/api/v3/btcbrl/ticker'
urlApiNegocieCoins = 'https://broker.negociecoins.com.br/api/v3/btcbrl/ticker'
urlApiNox = 'https://api.nox.trading/ticker/1'

In [112]:
data_static_NegocieCoins = {
    'deposit_brl_min': 50.0,
    'deposit_brl_max_day': 1000000.0,
    'deposit_brl_max_month': 0.0,
    'deposit_btc_min': 0.0001,
    'deposit_btc_max_day': 50.0,
    'deposit_btc_max_month': 500.0,
    'withdrawal_brl_min': 50.0,
    'withdrawal_brl_max_day': 30000.0,
    'withdrawal_brl_max_month': 720000.0,
    'withdrawal_btc_min': 0.0005,
    'withdrawal_btc_max_day': 500.0,
    'withdrawal_btc_max_month': 0.0,
    'fee_withdrawal_brl': '0.9%+8.90',
    'fee_withdrawal_btc': 0.0,
    'fee_deposit_brl': 0.0,
    'fee_deposit_btc': 0.0,
    'fee_withdrawal_fortknox_brl': 0.005,
    'fee_deposit_fortknox_brl': 0.0,
}

data_static_TemBTC = {
    'deposit_brl_min': 40.0,
    'deposit_brl_max_day': 300.0,
    'deposit_brl_max_day_premium': 2000.0,
    'deposit_brl_max_month': 0.0,
    'deposit_btc_min': 0.0001,
    'deposit_btc_max_day': 0.5,
    'deposit_btc_max_day_premium': 500.0,
    'deposit_btc_max_month': 0.0,
    'withdrawal_brl_min': 50.0,
    'withdrawal_brl_max_day': 300.0,
    'withdrawal_brl_max_day_premium': 30000.0,
    'withdrawal_brl_max_month': 0.0,
    'withdrawal_btc_min': 0.001,
    'withdrawal_btc_max_day': 0.5,
    'withdrawal_btc_max_day_premium': 500.0,
    'withdrawal_btc_max_month': 0.0,
    'fee_withdrawal_brl': '0.01+9.90',
    'fee_withdrawal_btc': 0.0003,
    'fee_deposit_brl': 0.0,
    'fee_deposit_btc': 0.0,
    'fee_withdrawal_fortknox_brl': 0.005,
    'fee_deposit_fortknox_brl': 0.0,
}

In [120]:
'''
@return APIResponse Resposta em json da API
'''
def requestData(url):
    headers = {'Content-Type': 'application/json'}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        #Requisição efetuada com sucesso
        return response.json()
    else:
        #Houve um erro
        return False

'''
@return APIResponse Resposta em json da API
'''
def getPrices():
    response_TemBTC = requestData(urlApiTemBTC)
    print("API TemBTC: \n", response_TemBTC)
    response_NegocieCoins = requestData(urlApiNegocieCoins)
    print("API NegocieCoins: \n", response_NegocieCoins)
    response_Nox = requestData(urlApiNox)
    print("API Nox:", response_Nox)
    return response_TemBTC, response_NegocieCoins, response_Nox

def calculateArbitrage(exchange_max, exchange_min, balance):
    #### Passo a passo
    #--- Taxa do saque da exchange_max (BRL)
    #>>> Valor em BRL
    first_fee_withdrawal = exchange_max*0.005
    #--- Saldo do saque após taxa de exchange_max para exchange_min
    first_withdrawal = exchange_max-first_fee_withdrawal
    #--- Taxa da execução de ordem ATIVA da exchange_min (BRL para BTC)
    #>>> Valor em BTC
    brl_to_btc = (first_withdrawal/exchange_min)
    first_fee_order_executed = brl_to_btc*(0.005)
    #--- Saldo da exchange_min após taxa de execução de ordem ATIVA
    #>>> Valor em BTC
    balance_btc = brl_to_btc-first_fee_order_executed
    #--- Taxa do saque da exchange_min (BTC)
    #>>> Valor em BTC
    second_fee_withdrawal = 0.0003
    second_withdrawal = balance_btc-second_fee_withdrawal
    #--- Taxa da execução de ordem ATIVA da exchange_max (BTC para BRL)
    #>>> Valor em BRL
    btc_to_brl = (second_withdrawal*exchange_max)
    second_fee_order_executed = btc_to_brl*(0.005)
    #>>> Balanço final após execução de ordem ATIVA da exchange_max
    final_balance_brl = btc_to_brl-second_fee_order_executed
    
    #--- Spread = Diferença de preço entre as exchanges
    #>>> Valor em porcentagem
    spread = ((exchange_max-exchange_min)/exchange_max)*100
    #>>> Taxa de Lucro
    rate_profit = (final_balance_brl/balance)-1
    profit_brl = final_balance_brl-balance
    
    #Print
    print("-------- Tabela Arbitragem --------\n")
    df_data = {
    'Operação':
    [
        'Cotação Exchange_MAX.:',
        'Cotação Exchange_MIN.:',
        'Saldo inicial:',
        'Taxa do primeiro saque:',
        'Valor transferido:',
        'Balanço BTC (Sem taxas):',
        'Taxa de ordem executada:',
        'Balanço BTC (Após taxas):',
        'Taxa do segundo saque:',
        'Valor transferido:',
        'Balanço BRL (Sem taxas):',
        'Taxa de ordem executada:',
        'Balanço BRL (Após taxas):',
        'Saldo Final:',
        '------ Spread ------>',
        '--- Taxa de Lucro --->',
        'Lucro Líquido:'
    ],
    'Preço':
    [
        '{0:.2f}'.format(exchange_max),
        '{0:.2f}'.format(exchange_min),
        '{0:.2f}'.format(balance),
        '{0:.2f}'.format(first_fee_withdrawal),
        '{0:.2f}'.format(first_withdrawal),
        '{0:.8f}'.format(brl_to_btc),
        '{0:.8f}'.format(first_fee_order_executed),
        '{0:.8f}'.format(balance_btc),
        '{0:.8f}'.format(second_fee_withdrawal),
        '{0:.8f}'.format(second_withdrawal),
        '{0:.2f}'.format(btc_to_brl),
        '{0:.2f}'.format(second_fee_order_executed),
        '{0:.2f}'.format(final_balance_brl),
        '{0:.2f}'.format(final_balance_brl),
        '{0:.2f}'.format(spread),
        '{0:.2f}'.format(rate_profit),
        '{0:.2f}'.format(profit_brl)
    ],
    'Tipo':
    [
        'BRL',
        'BRL',
        'BRL',
        'BRL',
        'BRL',
        'BTC',
        'BTC',
        'BTC',
        'BTC',
        'BTC',
        'BRL',
        'BRL',
        'BRL',
        'BRL',
        '%  ',
        '%  ',
        'BRL'
    ]}
    
    df = pd.DataFrame(df_data)
    print(df)
    print("\n-------------Done!-------------------")

# Código Main

In [115]:
temBTC, NegocieCoins, NoxTrading = getPrices()

API TemBTC: 
 {'buy': 21167.19, 'date': 1556598138, 'high': 21420.29, 'last': 21167.2, 'low': 21108.31, 'sell': 21167.2, 'vol': 81236.74583894}
API NegocieCoins: 
 {'buy': 21531.53, 'date': 1556598142, 'high': 21950.0, 'last': 21531.53, 'low': 21198.41, 'sell': 21531.54, 'vol': 181578.9187106}
API Nox: {'volume': 153.97, 'compra': 20466.95, 'venda': 20690.07, 'ultimo': 20660.0}


In [121]:
saldo = 59369.51
calculateArbitrage(NegocieCoins['last'],temBTC['last'],saldo)

-------- Tabela Arbitragem --------

                     Operação       Preço Tipo
0      Cotação Exchange_MAX.:    21531.53  BRL
1      Cotação Exchange_MIN.:    21167.20  BRL
2              Saldo inicial:    59369.51  BRL
3     Taxa do primeiro saque:      107.66  BRL
4          Valor transferido:    21423.87  BRL
5    Balanço BTC (Sem taxas):  1.01212595  BTC
6    Taxa de ordem executada:  0.00506063  BTC
7   Balanço BTC (Após taxas):  1.00706532  BTC
8      Taxa do segundo saque:  0.00030000  BTC
9          Valor transferido:  1.00676532  BTC
10   Balanço BRL (Sem taxas):    21677.20  BRL
11   Taxa de ordem executada:      108.39  BRL
12  Balanço BRL (Após taxas):    21568.81  BRL
13               Saldo Final:    21568.81  BRL
14      ------ Spread ------>        1.69  %  
15     --- Taxa de Lucro --->       -0.64  %  
16             Lucro Líquido:   -37800.70  BRL

-------------Done!-------------------
